In [9]:
%pip install holidays pandas numpy tabulate rich

  Using cached rich-14.2.0-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached rich-14.2.0-py3-none-any.whl (243 kB)
Using cached markdown_it_py-4.0.0-py3-none-any.whl (87 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [rich]2/3 [rich]
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import holidays
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.progress import Progress, SpinnerColumn, TextColumn

console = Console()

# ------------------------------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------------------------------
US_HOLIDAYS = holidays.US()

HOT_THRESHOLD = 80   # °F
COLD_THRESHOLD = 25  # °F

PANEL_PATH = "ticket-prophet/data/processed/weather _opt/TS_daily_panel_2025.csv"
WEATHER_PATH = "ticket-heroes/ticket-prophet/data/external/weather_master.csv"
OUTPUT_PATH = "ticket-heroes/ticket-prophet/data/processed/weather_opt/TS_daily_panel_2025_enriched.csv"


# ------------------------------------------------------------------------------------
# LOAD DATA
# ------------------------------------------------------------------------------------
def load_daily_panel(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df["date"] = pd.to_datetime(df["date"]).dt.date
    return df


def load_weather(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df["date"] = pd.to_datetime(df["time"]).dt.date
    return df


# ------------------------------------------------------------------------------------
# FEATURE ENGINEERING — WEATHER FLAGS
# ------------------------------------------------------------------------------------
def add_weather_flags(df_weather: pd.DataFrame) -> pd.DataFrame:
    # Fill missing values gracefully
    df_weather["precipitation"] = df_weather["precipitation"].fillna(0)
    df_weather["temp_max"] = df_weather["temp_max"].fillna(df_weather["temp_max"].median())
    df_weather["temp_min"] = df_weather["temp_min"].fillna(df_weather["temp_min"].median())

    # Weather flags
    df_weather["is_rain"] = df_weather["precipitation"] > 0
    df_weather["is_heavy_rain"] = df_weather["precipitation"] >= 0.5
    df_weather["is_hot"] = df_weather["temp_max"] >= HOT_THRESHOLD
    df_weather["is_cold"] = df_weather["temp_min"] <= COLD_THRESHOLD

    # Optional columns
    df_weather["is_windy"] = df_weather.get("wind_speed_max", pd.Series(False, index=df_weather.index)) >= 15
    df_weather["is_storm"] = df_weather.get("weather_code", pd.Series(None, index=df_weather.index)).isin([95, 96, 99])

    # Collapse to daily level (in case weather is hourly)
    weather_daily = df_weather.groupby("date")[[
        "is_rain",
        "is_heavy_rain",
        "is_hot",
        "is_cold",
        "is_windy",
        "is_storm"
    ]].max().reset_index()

    return weather_daily


# ------------------------------------------------------------------------------------
# FEATURE ENGINEERING — WEEKEND + HOLIDAY
# ------------------------------------------------------------------------------------
def add_calendar_flags(df: pd.DataFrame) -> pd.DataFrame:
    df["date_dt"] = pd.to_datetime(df["date"])

    df["is_weekend"] = df["date_dt"].dt.weekday >= 5
    df["is_holiday"] = df["date_dt"].isin(US_HOLIDAYS)

    df.drop(columns=["date_dt"], inplace=True)
    return df


# ------------------------------------------------------------------------------------
# MERGE + ORCHESTRATION
# ------------------------------------------------------------------------------------
def enrich_panel(panel_path: str, weather_path: str, output_path: str):
    with Progress(
        SpinnerColumn(),
        TextColumn("[bold blue]{task.description}"),
        transient=True
    ) as progress:

        t1 = progress.add_task("Loading panel...", start=True)
        panel = load_daily_panel(panel_path)
        progress.update(t1, description="Panel loaded ✓", completed=True)

        t2 = progress.add_task("Loading weather...", start=True)
        weather = load_weather(weather_path)
        progress.update(t2, description="Weather loaded ✓", completed=True)

        t3 = progress.add_task("Engineering weather flags...", start=True)
        weather_daily = add_weather_flags(weather)
        progress.update(t3, description="Weather flags complete ✓", completed=True)

        t4 = progress.add_task("Merging...", start=True)
        panel = panel.merge(weather_daily, on="date", how="left")
        progress.update(t4, description="Merge complete ✓", completed=True)

        t5 = progress.add_task("Adding weekend + holiday flags...", start=True)
        panel = add_calendar_flags(panel)
        progress.update(t5, description="Flags added ✓", completed=True)

        t6 = progress.add_task("Saving output...", start=True)
        panel.to_csv(output_path, index=False)
        progress.update(t6, description=f"Saved ✓ → {output_path}", completed=True)

    # Pretty summary table
    tbl = Table(title="Enrichment Summary", show_header=True, header_style="bold magenta")
    tbl.add_column("Metric", style="cyan")
    tbl.add_column("Value", style="green")

    tbl.add_row("Rows", str(len(panel)))
    tbl.add_row("Columns", str(len(panel.columns)))
    tbl.add_row("Output Path", output_path)

    console.print(Panel(tbl, title="[bold green]SUCCESS", border_style="green"))
    return panel


# ------------------------------------------------------------------------------------
# RUN
# ------------------------------------------------------------------------------------
if __name__ == "__main__":
    console.print(Panel("[bold yellow]🌤️ Beginning Daily Panel Weather Enrichment...", border_style="yellow"))
    enrich_panel(PANEL_PATH, WEATHER_PATH, OUTPUT_PATH)
    console.print(Panel("[bold green]🚀 Done! Panel enriched successfully.", border_style="green"))


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🌤️ Beginning Daily Panel Weather Enrichment...                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

KeyError: 'date'